# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

***
*WORK IN PROGRESS*
***

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

### --PENDING-- US County - Alternative: NY Times
- Link: https://github.com/nytimes/covid-19-data/
- **Historic time series data:**
    - US States (cases, deaths) --PENDING--
    - US Counties (cases, deaths) --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-12-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-11-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-11 22:45:33,34.223334,-82.461707,9,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-11 22:45:33,30.295065,-92.414197,98,4,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-11 22:45:33,37.767072,-75.632346,15,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-11 22:45:33,43.452658,-116.241552,513,6,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-11 22:45:33,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,273,281,299,349,367,423,444,484,521,555
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,277,304,333,361,377,383,400,409,416,433
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,986,1171,1251,1320,1423,1468,1572,1666,1761,1825


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,6,6,7,7,11,14,14,15,15,18
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,16,17,20,20,21,22,22,23,23,23
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,86,105,130,152,173,193,205,235,256,275


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,10,10,10,15,18,18,29,32,32,32
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,76,89,99,104,116,131,154,165,182,197
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,61,62,90,90,90,113,237,347,405,460


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,82,84,93,112,113,121,121,128,130,133
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,6,6,6,6,6,6,6,6,6,6


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,3,4,4,4,4,4,4,4,4,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,1,1,1,1,1,1,1,1,1,1


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
459,1027.0,Clay,Alabama,US,2020-04-11 22:45:33,33.269842,-85.858361,12,0,0,0,"Clay, Alabama, US"
426,1023.0,Choctaw,Alabama,US,2020-04-11 22:45:33,32.022273,-88.265644,8,0,0,0,"Choctaw, Alabama, US"
506,1033.0,Colbert,Alabama,US,2020-04-11 22:45:33,34.698475,-87.801685,8,1,0,0,"Colbert, Alabama, US"


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,526396,20463,31270
Spain,163027,16606,59109
Italy,152271,19468,32534


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,526396,20463,31270,474663,2020-04-11 22:45:33,38.044617,-90.642345
Spain,163027,16606,59109,87312,2020-04-11 22:45:13,40.463667,-3.749220
Italy,152271,19468,32534,100269,2020-04-11 22:45:13,41.871940,12.567380
France,130727,13851,26663,90213,2020-04-11 22:45:13,8.314863,-21.695174
Germany,124908,2736,57400,64772,2020-04-11 22:45:13,51.165691,10.451526


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,181026,8627,0
New Jersey,58151,2183,0
Michigan,23605,1384,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,181026,8627,0,172399,2020-04-11 22:45:33,42.671593,-75.579694
New Jersey,58151,2183,0,55968,2020-04-11 22:45:33,40.286441,-74.622450
Michigan,23605,1384,0,22221,2020-04-11 22:45:33,43.882838,-85.072001
Massachusetts,22860,686,0,22174,2020-04-11 22:45:33,42.176400,-71.577050
Pennsylvania,21719,503,0,21216,2020-04-11 22:45:33,40.818078,-77.615318


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered
Province_State Admin2                                     
New York       New York City      98308    6367          0
               Nassau             22584     792          0
               Suffolk            20321     458          0
               Westchester        18729     461          0
Illinois       Cook               13417     455          0

In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

Confirmed  Deaths  Recovered  Active  \
Province_State Admin2                                                
New York       New York City      98308    6367          0   91941   
               Nassau             22584     792          0   21792   
               Suffolk            20321     458          0   19863   
               Westchester        18729     461          0   18268   
Illinois       Cook               13417     455          0   12962   

                                      Last_Update        Lat       Long  \
Province_State Admin2                                                     
New York       New York City  2020-04-11 22:45:33  40.767273 -73.971526   
               Nassau         2020-04-11 22:45:33  40.740665 -73.589419   
               Suffolk        2020-04-11 22:45:33  40.883201 -72.801217   
               Westchester    2020-04-11 22:45:33  41.162784 -73.757417   
Illinois       Cook           2020-04-11 22:45:33  41.841448 -87.816588   

                                 FIPS  
Province_State Admin2                  
New York       New York City  36061.0  
               Nassau         36059.0  
               Suffolk        36103.0  
               Westchester    36119.0  
Illinois       Cook           17031.0

##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-04-09,484,409,1666,583,19,19,1795,921,6108,13244,...,65872,456,582,171,255,263,4,0,39,11
2020-04-10,521,416,1761,601,19,19,1975,937,6215,13555,...,74605,473,624,171,257,267,4,1,40,13
2020-04-11,555,433,1825,601,19,21,1975,967,6303,13806,...,79874,494,767,175,258,268,4,1,40,14


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-04-09,15,23,235,25,2,2,72,10,51,295,...,7993,7,3,9,0,1,0,0,1,3
2020-04-10,15,23,256,26,2,2,82,12,54,319,...,8974,7,3,9,0,2,0,0,2,3
2020-04-11,18,23,275,26,2,2,83,13,57,337,...,9892,7,4,9,0,2,0,0,2,3


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-04-09,32,165,347,58,2,0,365,138,1472,5240,...,359,192,38,84,128,44,0,0,24,0
2020-04-10,32,182,405,71,2,0,375,149,1793,6064,...,588,206,42,84,144,45,0,0,25,0
2020-04-11,32,197,460,71,4,0,440,173,1806,6604,...,622,214,42,93,144,57,0,0,28,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
3149,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,6,16,17,20,33,39,42,60,69,82
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,11,16,17,17,19,21,23,33,39,42
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,5,7,8,8,13,14,14,14,15,19
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,3,5,5,5,5,6,6,9,9,11


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-04-09,2703,235,0,3018,1119,19710,5655,9784,1209,49,...,4634,11208,1856,628,45,4042,9639,514,2886,230
2020-04-10,2947,246,0,3112,1171,21081,6202,10538,1326,49,...,4891,12105,2103,679,50,4509,10119,574,3068,239
2020-04-11,3217,257,0,3393,1228,21706,6513,11510,1479,49,...,5132,13023,2207,711,51,5077,10434,577,3213,261


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-04-09          15      56       4    9     11       3      3      53   
2020-04-10          17      59       9   11     12       4      3      54   
2020-04-11          19      66       9   13     12       4      6      57   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-04-09          151        7  ...       1      0       12        1   
2020-04-10          168        7  ...       1      0       12        1   
2020-04-11          181        7  ...       1      0       12        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-04-09              6    45     3          0        5      0  
2020-04-10              6    50     4          0        5      0  
2020-04-11              7    53     4          0        5      0  

[3 rows x 3246 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-04-09,70,7,0,89,21,544,193,380,23,0,...,94,210,13,23,1,109,456,5,111,0
2020-04-10,80,7,0,97,21,583,226,448,32,0,...,98,238,13,24,1,121,483,5,128,0
2020-04-11,92,8,0,108,25,605,250,494,33,0,...,106,266,13,25,1,130,495,5,137,0


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-04-09           1       1       0    0      0       0      0       0   
2020-04-10           1       1       0    0      0       0      0       0   
2020-04-11           1       1       0    0      0       0      0       0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-04-09            8        0  ...       0      0        0        0   
2020-04-10            8        0  ...       0      0        0        0   
2020-04-11            8        0  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-04-09              0     0     0          0        0      0  
2020-04-10              0     0     0          0        0      0  
2020-04-11              0     0     0          0        0      0  

[3 rows x 3246 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  

In [54]:
states_curr = pd.read_csv('https://covidtracking.com/api/states.csv', index_col='state')

In [55]:
states_hist = pd.read_csv('http://covidtracking.com/api/states/daily.csv')

In [56]:
us_curr = pd.read_csv('https://covidtracking.com/api/us.csv')
us_curr

,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,hash,lastModified,death,hospitalized,total,totalTestResults,posNeg,notes
0,551826,2254066,16419,51413,61201,13917,1455,5986,160,34151,fab18071448220868a8bb0a6920c31fe4bdf5624,2020-04-12T21:12:06.554Z,21919,61201,2822311,2805892,2805892,"NOTE: ""total"", ""posNeg"", ""hospitalized"" will b..."


In [57]:
us_hist = pd.read_csv('https://covidtracking.com/api/us/daily.csv')

### Dataset 3. US Counties - New York Times
https://github.com/nytimes/covid-19-data/

In [58]:
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [59]:
counties.date = pd.to_datetime(counties.date)

In [60]:
nytdate = counties.date.max()
nytdate

Timestamp('2020-04-11 00:00:00')

In [61]:
counties.set_index('date', inplace=True)
counties.head()

,county,state,fips,cases,deaths
date,,,,,
2020-01-21,Snohomish,Washington,53061.0,1,0
2020-01-22,Snohomish,Washington,53061.0,1,0
2020-01-23,Snohomish,Washington,53061.0,1,0
2020-01-24,Cook,Illinois,17031.0,1,0
2020-01-24,Snohomish,Washington,53061.0,1,0


## Data Wrangling

Get the data into a format where it's machine readable for analytics and forecasting

In [62]:
# Covid Tracking Project - US State data.  Look at first 5 rows before reformatting
states_hist.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200412,AK,272.0,7766.0,NaN,NaN,31.0,NaN,NaN,NaN,...,31.0,8038.0,8038.0,8038.0,2,0.0,0.0,291.0,15.0,306.0
1,20200412,AL,3525.0,18058.0,NaN,NaN,437.0,NaN,189.0,NaN,...,437.0,21583.0,21583.0,21583.0,1,2.0,35.0,0.0,334.0,334.0
2,20200412,AR,1280.0,18442.0,NaN,74.0,130.0,NaN,43.0,30.0,...,130.0,19722.0,19722.0,19722.0,5,3.0,0.0,1090.0,54.0,1144.0
3,20200412,AS,0.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200412,AZ,3539.0,38570.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,42109.0,42109.0,42109.0,4,7.0,0.0,1433.0,146.0,1579.0


In [63]:
# change YYYYMMDD to YYYY-MM-DD and convert to datetime

states_hist['date'] = states_hist['date'].apply(lambda x : str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:8])
states_hist['date'] = states_hist['date'].apply(lambda x : pd.to_datetime(x) )

In [64]:
# Now set the date as the index

states_hist.set_index('date', inplace=True)
states_hist.sort_index(inplace=True)

In [65]:
states_hist.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
date,,,,,,,,,,,,,,,,,,,,,
2020-02-28,WA,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.0,9.0,9.0,53,NaN,NaN,NaN,NaN,NaN
2020-02-29,WA,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.0,18.0,18.0,53,1.0,0.0,0.0,9.0,9.0
2020-03-01,RI,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,44,NaN,NaN,NaN,NaN,NaN
2020-03-01,WA,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30.0,30.0,30.0,53,3.0,0.0,0.0,12.0,12.0
2020-03-02,WA,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,34.0,34.0,34.0,53,3.0,0.0,0.0,4.0,4.0


In [66]:
# get the latest date so we can pull recent data as needed
ctp_date = states_hist.index.max()
ctp_date

Timestamp('2020-04-12 00:00:00')

In [67]:
# get a list of the top 15 states using current data
top_states = states_curr['positive'].sort_values(ascending=False).nlargest(15).index
top_states

Index(['NY', 'NJ', 'MA', 'MI', 'PA', 'CA', 'IL', 'LA', 'FL', 'TX', 'GA', 'CT',
       'WA', 'MD', 'IN'],
      dtype='object', name='state')

In [68]:
# Now create a pivot tables for positive, negative, pending, hospitalized, death, total

states_positive = pd.pivot_table(states_hist, index='date', columns='state', values='positive')
states_negative = pd.pivot_table(states_hist, index='date', columns='state', values='negative')
states_pending = pd.pivot_table(states_hist, index='date', columns='state', values='pending')
states_hospitalized = pd.pivot_table(states_hist, index='date', columns='state', values='hospitalized')
states_deaths = pd.pivot_table(states_hist, index='date', columns='state', values='death')
states_tests = pd.pivot_table(states_hist, index='date', columns='state', values='total')

## Start examining the data - Exploratory Data Analysis (EDA)

In [69]:
# Total Global confirmed cases
jh_live_global['Confirmed'].sum()

1771514

In [70]:
# Total Global deaths
jh_live_global['Deaths'].sum()

108503

In [71]:
# Total US confirmed cases
jh_live_global[jh_live_global["Country_Region"]=='US']['Confirmed'].sum()

526396

In [72]:
# Total US Deaths (per JH dataset)
jh_live_global[jh_live_global["Country_Region"]=='US']['Deaths'].sum()

20463

Make a Pivot Table to summarize the global confirmed cases by Country  
**DONE**  
date_columns = global_confirmed.iloc[:, 4:].columns
confirmed_country = pd.pivot_table(global_confirmed, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
confirmed_country = confirmed_country.transpose()
confirmed_country.index = pd.to_datetime(confirmed_country.index)
confirmed_country = confirmed_country.sort_index()

In [73]:
counties_CA_filter = counties['state']=='California'
counties_CA_cases = pd.pivot_table(counties[counties_CA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [74]:
top_CA_counties = counties_CA_cases.loc[nytdate, :].nlargest(10).index
top_CA_counties

Index(['Los Angeles', 'San Diego', 'Santa Clara', 'Riverside', 'Orange',
       'San Francisco', 'Alameda', 'San Bernardino', 'Sacramento',
       'San Mateo'],
      dtype='object', name='county')

In [75]:
counties_MA_filter = counties['state']=='Massachusetts'
counties_MA_cases = pd.pivot_table(counties[counties_MA_filter], index='date', columns='county', values='cases', aggfunc=np.sum)

In [76]:
top_MA_counties = counties_MA_cases.loc[nytdate, :].nlargest(10).index
top_MA_counties

Index(['Suffolk', 'Middlesex', 'Essex', 'Norfolk', 'Worcester', 'Plymouth',
       'Hampden', 'Bristol', 'Barnstable', 'Berkshire'],
      dtype='object', name='county')